In [1]:
#Recommendation system

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Sample data (user ratings for movies)
data = {
    'User': ['User1', 'User2', 'User3', 'User4', 'User5'],
    'Movie1': [5, 4, 0, 2, 1],
    'Movie2': [4, 5, 1, 3, 2],
    'Movie3': [0, 2, 5, 4, 3],
    'Movie4': [3, 1, 4, 5, 0],
    'Movie5': [2, 3, 0, 1, 5],
}

df = pd.DataFrame(data)

# Function to calculate cosine similarity between users
def calculate_similarity(dataframe):
    user_similarity = cosine_similarity(dataframe.iloc[:, 1:])
    return pd.DataFrame(user_similarity, columns=dataframe['User'], index=dataframe['User'])

# Function to get movie recommendations for a given user
def get_movie_recommendations(user, similarity_matrix, dataframe, num_recommendations=3):
    user_ratings = dataframe[dataframe['User'] == user].iloc[:, 1:]
    similar_users = similarity_matrix[user].sort_values(ascending=False).index[1:]
    
    recommendations = []

    for movie in user_ratings.columns:
        if user_ratings[movie].values[0] == 0:  # Not yet rated
            total_similarity = 0
            weighted_sum = 0

            for similar_user in similar_users:
                sim_score = similarity_matrix[user][similar_user]
                weighted_sum += sim_score * dataframe[dataframe['User'] == similar_user][movie].values[0]
                total_similarity += abs(sim_score)

            if total_similarity > 0:
                predicted_rating = weighted_sum / total_similarity
                recommendations.append((movie, predicted_rating))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:num_recommendations]

# Split data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Calculate user similarity
user_similarity_matrix = calculate_similarity(train_data)

# Example: Get movie recommendations for User1
user_recommendations = get_movie_recommendations('User1', user_similarity_matrix, train_data)
print(f"Recommended movies for User1: {user_recommendations}")
